In [4]:
import random
import pandas
import numpy as np

In [5]:
def get_params():
  n = int(input('Введите количество устройств n'))
  m = int(input('Введите количество задач m'))
  r1, r2 = [int(x) for x in input('Введите левую и правую границы генерации r1 r2').split(' ')]
  k = int(input('Введите количество повторов k'))
  n_ind = int(input('Введите количество особей в поколении Ind'))
  p_cross = float(input('Введите вероятность кроссовера Pk'))
  p_mut = float(input('Введите вероятность мутации Pm'))
  model_type = int(input('Введите модель генетического алгоритма (1. Голберг 2. Холланд)'))
  
  if model_type == 1:
    return n, np.random.randint(r1, r2, m).tolist(), p_cross, p_mut, n_ind,  k, model_type
  else:
    return n, np.random.randint(r1, r2, (m, n)).tolist(), p_cross, p_mut, n_ind, k, model_type

In [110]:
class Individual():

  def __init__(self, n, task_matrix, p_mut, p_cross, num, model):
      self.model = model
      self.n = n
      self.x_gens = task_matrix
      self.y_gens = np.random.randint(0, 255, len(task_matrix)).tolist()
      self.mut_prop = [*[1 for i in range(int(p_mut * 100))], *[0 for i in range(100 - int(p_mut * 100))]]
      self.cross_prop = [*[1 for i in range(int(p_cross * 100))], *[0 for i in range(100 - int(p_cross * 100))]]
      self.p_mut = p_mut
      self.p_cross = p_cross
      self.phenotype = self.get_pheno()
      self.num = num

  def get_genotype(self):
      self.get_genotype = [(x, y) for x in self.x_gens for y in self.y_gens]

  def print_info(self):
        print(f"Генотип -->")
        print("\t" + " ".join(map(str, self.x_gens)))
        print("\t" + " ".join(map(str, self.y_gens)))
        self.print_pheno()
        print("\n")

  def get_pheno(self):
    interval_size = 256 // self.n
    ranges = []
    phenos = {}

    start = 0
    for i in range(1, self.n+1):
        end = 255 - (self.n - i) * interval_size
        ranges.append((start, end))
        phenos[f'{start}...{end}'] = []
        start = end + 1
    
    for gen_num in range(len(self.y_gens)):
        for range_num in range(len(ranges)):
            r = ranges[range_num]
            if r[0] <= self.y_gens[gen_num] < r[1]:
                if self.model == 1:
                    phenos[f'{r[0]}...{r[1]}'].append(self.x_gens[gen_num])
                else:
                    print(self.x_gens, r)
                    print(gen_num, range_num)
                    print(ranges)
                    phenos[f'{r[0]}...{r[1]}'].append(self.x_gens[gen_num][range_num])
                
    max_feno_num = np.argmax([sum(values) for values in phenos.values()])
    max_feno = list(phenos.values())[max_feno_num]
    
    return {
        'ranges': ranges,
        'phenos': phenos,
        'max_feno': max_feno,
        'max_feno_num': max_feno_num
    }
    
  def update_pheno(self):
    self.phenotype = self.get_pheno()

  def print_pheno(self):
    print(f"Фенотип:")
    for num, (pheno, values) in enumerate(self.phenotype['phenos'].items()):
        sum_values = sum(values)
        max_indicator = "<-- MAX" if num == self.phenotype['max_feno_num'] else ""
        print(f"{num+1}) {pheno} : {values} | Сумма: {sum_values} {max_indicator}")

  def mutation(self):
    if self.mut_prop[random.randint(0, 99)] == 1:
        print(f"Произошла мутация (её шанс был {self.mut_prop.count(1) / 100})", end='\n')
        
        random_gene_index = random.randint(0, len(self.y_gens) - 1)
        random_gene = self.y_gens[random_gene_index]

        if self.model == 2:
            bit_length = random_gene.bit_length()
            while bit_length < 2:
                random_gene = self.y_gens[random.randint(0, len(self.y_gens) - 1)]
                bit_length = random_gene.bit_length()

        
            random_bit_positions = random.sample(range(bit_length), 2)
            mask1 = 1 << random_bit_positions[0]
            mask2 = 1 << random_bit_positions[1]
            mutated_gene = random_gene ^ mask1 ^ mask2
        else:
            bit_length = random_gene.bit_length()
            while bit_length == 0:
                random_gene = self.y_gens[random.randint(0, len(self.y_gens) - 1)]
                bit_length = random_gene.bit_length()

            random_bit_position = random.randint(0, bit_length - 1)
            mask = 1 << random_bit_position
            mutated_gene = random_gene ^ mask

        self.y_gens[random_gene_index] = mutated_gene
    else:
        print("Мутация не произошла", end='\n')

        
  def crossover(self, ind: 'Individual'):
      if self.cross_prop[random.randint(0, 99)] == 1:
        first_child = Individual(ind.n, ind.x_gens, ind.p_mut, ind.p_cross, num=self.num, model=self.model)
        second_child = Individual(ind.n, ind.x_gens, ind.p_mut, ind.p_cross, num=self.num, model=self.model)
        
        if self.model == 1:
            split_dot = np.random.randint(1, len(self.y_gens) - 1)
            first_child.y_gens = self.y_gens[:split_dot] + ind.y_gens[split_dot:]
            second_child.y_gens = ind.y_gens[:split_dot] + self.y_gens[split_dot:]
        else:
            left_split_dot = np.random.randint(1, len(self.y_gens) - 1)
            right_split_dot = np.random.randint(1, len(self.y_gens) - 1)
            
            first_child.y_gens = self.y_gens[:left_split_dot] + ind.y_gens[left_split_dot:right_split_dot] + self.y_gens[right_split_dot:]
            second_child.y_gens = ind.y_gens[:left_split_dot] + self.y_gens[left_split_dot:right_split_dot] + ind.y_gens[right_split_dot:]
        """
        print(f"Точка разрыва: {split_dot}")
        print(f"Гены родителя 1 ребенка 1: {self.y_gens[:split_dot]}")
        print(f"Гены родителя 2 ребенка 1: {ind.y_gens[split_dot:]}")
        print(f"Гены родителя 1 ребенка 2: {ind.y_gens[split_dot:]}")
        print(f"Гены родителя 2 ребенка 2: {self.y_gens[:split_dot]}")
        """

        print(f"Прошёл кроссовер (его шанс был {self.cross_prop.count(1) / 100})", end='\n')
        
        childs = [first_child, second_child]
        [child.update_pheno() for child in childs]
        #result_num = np.argmax(np.array[first_child.get_pheno, second_child.get_pheno, self.get_pheno])

        #return childs[result_num]
        return childs
      else:
        print("Кроссовер не произошёл", end='\n')
        return None

In [13]:
class Evolution:
    def __init__(self, n, task_vector, p_mut, p_cross, population_size = 5, k = 3, model=1) -> None:
        self.population_size = population_size
        self.population = [Individual(n, task_vector, p_mut, p_cross, i+1, model) for i in range(self.population_size)]
        self.num_generations = k
        if model == 1:
            self.generation_buffer = {}
        else:
            self.generation_buffer = []
        self.model = model
        
    def get_random_individual(self, current_ind):
        while True:
            random_ind = random.choice(self.population)
            if random_ind != current_ind:
                return random_ind
    
    def print_population(self):
        for ind in self.population:
            print(f"Особь {self.population.index(ind) + 1}")
            ind.print_info()
            
    def get_childs(self, ind1, ind2):
        childs = ind1.crossover(ind2)
        if childs is not None:
            
            for ind in childs:
                ind.mutation()
                ind.update_pheno()
                
            ind1.update_pheno()
            childs.append(ind1)
            return childs
        else:
            mut_ind = Individual(ind1.n, ind1.x_gens, ind1.p_mut, ind1.p_cross, num=ind1.num, model=ind1.model)
            mut_ind.y_gens = ind1.y_gens.copy()
            mut_ind.mutation()
            return [ind1, mut_ind]
        
    def get_best_individual(self, individuals):
        return min(individuals, key=lambda x: sum(x.phenotype['max_feno']))

    def get_top_in_generation(self, individuals):
        return sorted(individuals, key=lambda x: x.phenotype['max_feno'])[:self.population_size]

In [31]:
params = get_params()
params

(3,
 [[12, 15, 13],
  [13, 17, 10],
  [16, 16, 15],
  [13, 13, 16],
  [16, 13, 17],
  [15, 17, 15],
  [11, 15, 13]],
 0.5,
 0.5,
 5,
 3,
 2)

In [111]:
ind1 = Individual(7, np.random.randint(1, 10, (3, 7)).tolist(), 0.5, 0.5, 1, model=2)
ind2 = Individual(7, np.random.randint(1, 10, (3, 7)).tolist(), 0.5, 0.5, 2, model=2)
ind1.print_info()
ind2.print_info()

[[9, 9, 1, 6, 2, 1, 5], [5, 9, 5, 1, 5, 2, 1], [5, 3, 9, 1, 5, 8, 7]] (0, 39)
0 0
[(0, 39), (40, 75), (76, 111), (112, 147), (148, 183), (184, 219), (220, 255)]
[[9, 9, 1, 6, 2, 1, 5], [5, 9, 5, 1, 5, 2, 1], [5, 3, 9, 1, 5, 8, 7]] (40, 75)
1 1
[(0, 39), (40, 75), (76, 111), (112, 147), (148, 183), (184, 219), (220, 255)]
[[9, 9, 1, 6, 2, 1, 5], [5, 9, 5, 1, 5, 2, 1], [5, 3, 9, 1, 5, 8, 7]] (40, 75)
2 1
[(0, 39), (40, 75), (76, 111), (112, 147), (148, 183), (184, 219), (220, 255)]
[[4, 3, 2, 5, 7, 7, 7], [3, 4, 2, 2, 6, 5, 2], [3, 6, 4, 2, 1, 5, 5]] (148, 183)
0 4
[(0, 39), (40, 75), (76, 111), (112, 147), (148, 183), (184, 219), (220, 255)]
[[4, 3, 2, 5, 7, 7, 7], [3, 4, 2, 2, 6, 5, 2], [3, 6, 4, 2, 1, 5, 5]] (184, 219)
1 5
[(0, 39), (40, 75), (76, 111), (112, 147), (148, 183), (184, 219), (220, 255)]
[[4, 3, 2, 5, 7, 7, 7], [3, 4, 2, 2, 6, 5, 2], [3, 6, 4, 2, 1, 5, 5]] (184, 219)
2 5
[(0, 39), (40, 75), (76, 111), (112, 147), (148, 183), (184, 219), (220, 255)]
Генотип -->
	[9, 9, 1, 

In [142]:
result = evolution.get_childs(ind1, ind2)

[[4, 3, 2, 5, 7, 7, 7], [3, 4, 2, 2, 6, 5, 2], [3, 6, 4, 2, 1, 5, 5]] (184, 219)
0 5
[(0, 39), (40, 75), (76, 111), (112, 147), (148, 183), (184, 219), (220, 255)]
[[4, 3, 2, 5, 7, 7, 7], [3, 4, 2, 2, 6, 5, 2], [3, 6, 4, 2, 1, 5, 5]] (184, 219)
2 5
[(0, 39), (40, 75), (76, 111), (112, 147), (148, 183), (184, 219), (220, 255)]
[[4, 3, 2, 5, 7, 7, 7], [3, 4, 2, 2, 6, 5, 2], [3, 6, 4, 2, 1, 5, 5]] (0, 39)
0 0
[(0, 39), (40, 75), (76, 111), (112, 147), (148, 183), (184, 219), (220, 255)]
[[4, 3, 2, 5, 7, 7, 7], [3, 4, 2, 2, 6, 5, 2], [3, 6, 4, 2, 1, 5, 5]] (40, 75)
1 1
[(0, 39), (40, 75), (76, 111), (112, 147), (148, 183), (184, 219), (220, 255)]
[[4, 3, 2, 5, 7, 7, 7], [3, 4, 2, 2, 6, 5, 2], [3, 6, 4, 2, 1, 5, 5]] (40, 75)
2 1
[(0, 39), (40, 75), (76, 111), (112, 147), (148, 183), (184, 219), (220, 255)]
Прошёл кроссовер (его шанс был 0.5)
[[4, 3, 2, 5, 7, 7, 7], [3, 4, 2, 2, 6, 5, 2], [3, 6, 4, 2, 1, 5, 5]] (0, 39)
0 0
[(0, 39), (40, 75), (76, 111), (112, 147), (148, 183), (184, 219), (2

In [51]:
result[0].print_info()

Генотип -->
	[1, 7, 9, 1, 1, 2, 8] [7, 3, 7, 7, 9, 6, 3] [7, 9, 5, 4, 6, 1, 8]
	86 32 159
Фенотип:
1) 0...39 : [7] | Сумма: 7 
2) 40...75 : [] | Сумма: 0 
3) 76...111 : [9] | Сумма: 9 <-- MAX
4) 112...147 : [] | Сумма: 0 
5) 148...183 : [6] | Сумма: 6 
6) 184...219 : [] | Сумма: 0 
7) 220...255 : [] | Сумма: 0 




In [109]:
ind1.mutation()
ind1.print_info()

Произошла мутация (её шанс был 0.5)
Генотип -->
	[6, 3, 6, 8, 7, 8, 1] [9, 5, 6, 5, 8, 5, 4] [3, 2, 8, 5, 6, 8, 1]
	12 1 113
Фенотип:
1) 0...39 : [9] | Сумма: 9 <-- MAX
2) 40...75 : [] | Сумма: 0 
3) 76...111 : [6] | Сумма: 6 
4) 112...147 : [] | Сумма: 0 
5) 148...183 : [6] | Сумма: 6 
6) 184...219 : [] | Сумма: 0 
7) 220...255 : [] | Сумма: 0 




Генотип -->
	[6, 3, 6, 8, 7, 8, 1] [9, 5, 6, 5, 8, 5, 4] [3, 2, 8, 5, 6, 8, 1]
	86 8 18
Фенотип:
1) 0...39 : [9] | Сумма: 9 <-- MAX
2) 40...75 : [] | Сумма: 0 
3) 76...111 : [6] | Сумма: 6 
4) 112...147 : [] | Сумма: 0 
5) 148...183 : [6] | Сумма: 6 
6) 184...219 : [] | Сумма: 0 
7) 220...255 : [] | Сумма: 0 




In [70]:
ind2.crossover(ind1)

[[6, 3, 6, 8, 7, 8, 1], [9, 5, 6, 5, 8, 5, 4], [3, 2, 8, 5, 6, 8, 1]] (148, 183)
0 4
[(0, 39), (40, 75), (76, 111), (112, 147), (148, 183), (184, 219), (220, 255)]
[[6, 3, 6, 8, 7, 8, 1], [9, 5, 6, 5, 8, 5, 4], [3, 2, 8, 5, 6, 8, 1]] (76, 111)
1 2
[(0, 39), (40, 75), (76, 111), (112, 147), (148, 183), (184, 219), (220, 255)]
[[6, 3, 6, 8, 7, 8, 1], [9, 5, 6, 5, 8, 5, 4], [3, 2, 8, 5, 6, 8, 1]] (184, 219)
2 5
[(0, 39), (40, 75), (76, 111), (112, 147), (148, 183), (184, 219), (220, 255)]
[[6, 3, 6, 8, 7, 8, 1], [9, 5, 6, 5, 8, 5, 4], [3, 2, 8, 5, 6, 8, 1]] (76, 111)
0 2
[(0, 39), (40, 75), (76, 111), (112, 147), (148, 183), (184, 219), (220, 255)]
[[6, 3, 6, 8, 7, 8, 1], [9, 5, 6, 5, 8, 5, 4], [3, 2, 8, 5, 6, 8, 1]] (0, 39)
1 0
[(0, 39), (40, 75), (76, 111), (112, 147), (148, 183), (184, 219), (220, 255)]
[[6, 3, 6, 8, 7, 8, 1], [9, 5, 6, 5, 8, 5, 4], [3, 2, 8, 5, 6, 8, 1]] (220, 255)
2 6
[(0, 39), (40, 75), (76, 111), (112, 147), (148, 183), (184, 219), (220, 255)]
Прошёл кроссовер (его

In [143]:
evolution = Evolution(*params)
#evolution = Evolution(7, task_matrix, 0.5, 0.5, 5, model=1)
ostanova_counter = 0
last_pheno = 0
gen_counter = 0


print("Начальное распределение:")
evolution.print_population()
    
generation = 0
while True:
    print(f"Формирование нового {generation + 1}-го поколения: \n")
    for current_ind in evolution.population:
        partner_ind = evolution.get_random_individual(current_ind)

        print(f"Скрещивание особей {current_ind.num} и {partner_ind.num}")
        print(f"Особь {current_ind.num}")
        current_ind.print_info()

        print(f"Особь {partner_ind.num}")
        partner_ind.print_info()

        childs = evolution.get_childs(current_ind, partner_ind)
        best_ind = evolution.get_best_individual(childs)

        print(f"Лучшая особь среди потомков:")
        best_ind.print_info()

        if evolution.model == 1:
            evolution.generation_buffer[current_ind.num] = best_ind
        else:
            evolution.generation_buffer.extend(childs)

    print(f"Поколение {generation + 1} сформировано!")
    print(f"Лучшая особь текущего поколения:")
    best_in_gen = evolution.get_best_individual(evolution.generation_buffer.values())
    best_in_gen.print_info()

    if last_pheno == sum(best_in_gen.phenotype['max_feno']):
        ostanova_counter += 1
    else:
        last_pheno = sum(best_in_gen.phenotype['max_feno'])
        ostanova_counter = 0

    gen_counter += 1
    print(f"Количество поколений без изменений: {ostanova_counter}")

    if ostanova_counter >= evolution.num_generations:
        print("Количество поколений без изменений превысило заданное значение. Обучение завершено!")
        print("Прошло полных поколений: ", gen_counter)
        break

    evolution.population = list(evolution.generation_buffer.values())
    generation += 1

print("Обучение завершено!")

[[12, 15, 13], [13, 17, 10], [16, 16, 15], [13, 13, 16], [16, 13, 17], [15, 17, 15], [11, 15, 13]] (0, 85)
0 0
[(0, 85), (86, 170), (171, 255)]
[[12, 15, 13], [13, 17, 10], [16, 16, 15], [13, 13, 16], [16, 13, 17], [15, 17, 15], [11, 15, 13]] (0, 85)
1 0
[(0, 85), (86, 170), (171, 255)]
[[12, 15, 13], [13, 17, 10], [16, 16, 15], [13, 13, 16], [16, 13, 17], [15, 17, 15], [11, 15, 13]] (0, 85)
2 0
[(0, 85), (86, 170), (171, 255)]
[[12, 15, 13], [13, 17, 10], [16, 16, 15], [13, 13, 16], [16, 13, 17], [15, 17, 15], [11, 15, 13]] (0, 85)
3 0
[(0, 85), (86, 170), (171, 255)]
[[12, 15, 13], [13, 17, 10], [16, 16, 15], [13, 13, 16], [16, 13, 17], [15, 17, 15], [11, 15, 13]] (171, 255)
4 2
[(0, 85), (86, 170), (171, 255)]
[[12, 15, 13], [13, 17, 10], [16, 16, 15], [13, 13, 16], [16, 13, 17], [15, 17, 15], [11, 15, 13]] (0, 85)
5 0
[(0, 85), (86, 170), (171, 255)]
[[12, 15, 13], [13, 17, 10], [16, 16, 15], [13, 13, 16], [16, 13, 17], [15, 17, 15], [11, 15, 13]] (171, 255)
6 2
[(0, 85), (86, 170)

IndexError: list index out of range